In [17]:
using DataStructures
using BenchmarkTools

In [2]:
mutable struct Game
    player_points::Int64
    mana::Int64
    boss_points::Int64
    boss_damage::Int64
    armor::Int64
    shield_counter::Int64
    poison_counter::Int64
    recharge_counter::Int64
    missile_active::Bool
    drain_active::Bool
    turn::String
    shield_restart::Bool
    poison_restart::Bool
    recharge_restart::Bool
    part::Int64
    shield_wait::Bool
    poison_wait::Bool
    recharge_wait::Bool
end

In [3]:
Game(; player_points=50, mana=500, boss_points=58, boss_damage=9, part=1) = Game(player_points, mana, boss_points, boss_damage, 0, 0, 0, 0, false, false, "player", false, false, false, part, false, false, false)

Game

In [37]:
function state(g::Game)
    (g.player_points, g.mana, g.boss_points, g.armor, g.shield_counter, 
        g.poison_counter, g.recharge_counter, g.missile_active, g.drain_active, g.turn)
end

function activate_missile!(g::Game)
    g.missile_active = true
    g.mana -= 53
end

function activate_drain!(g::Game)
    g.drain_active = true
    g.mana -= 73
end
    
function activate_shield!(g::Game)
    if g.shield_counter == 0
        g.shield_wait = true
    else
        g.shield_wait = false
    end
    if g.shield_counter == 1
        g.shield_restart = true
    end
    g.shield_counter = 6
    g.mana -= 113
end

function activate_poison!(g::Game)
    if g.poison_counter == 0
        g.poison_wait = true
    else
        g.poison_wait = false
    end
    if g.poison_counter == 1
        g.poison_restart = true
    end
    g.poison_counter = 6
    g.mana -= 173
end
    
function activate_recharge!(g::Game)
    if g.recharge_counter == 0
        g.recharge_wait = true
    else
        g.recharge_wait = false
    end
    if g.recharge_counter == 1
        g.recharge_restart = true
    end
    g.recharge_counter = 5
    g.mana -= 229
end

function apply_effects!(g::Game)
    if g.missile_active
        g.boss_points -= 4
        g.missile_active = false
    end
    if g.drain_active
        g.boss_points -= 2
        g.player_points += 2
        g.drain_active = false
    end
    if g.shield_counter > 0
        if !g.shield_wait
            g.armor = 7
            if !g.shield_restart
                g.shield_counter -= 1
            else
                g.shield_restart = false
            end
        else
            g.shield_wait = false
        end
    else
        g.armor = 0
    end
    if g.poison_counter > 0
        if !g.poison_wait
            g.boss_points -= 3
            if !g.poison_restart
                g.poison_counter -= 1
            else
                g.poison_restart = false
            end
        else
            g.poison_wait = false
        end
    end
    if g.recharge_counter > 0
        if !g.recharge_wait
            g.mana += 101
            if !g.recharge_restart
                g.recharge_counter -= 1
            else
                g.recharge_restart = false
            end
        else
            g.recharge_wait = false
        end
    end
end

function go_turn!(g::Game)
    res = 0
    if g.part == 2 && g.turn == "player"
        g.player_points -= 1
    end
    apply_effects!(g)
    if g.boss_points <= 0
        res = 1
    end
    if g.turn == "boss"
        g.player_points -= max(g.boss_damage - g.armor, 1)
        if g.player_points <= 0 && res == 0
            res = -1
        end
        g.turn = "player"
    else
        g.turn = "boss"
    end
    res
end

function next_possible_states(g::Game)
    states = Tuple{Game, Int64}[]
    if g.mana >= 53
        h = deepcopy(g)
        activate_missile!(h)
        push!(states, (h, 53))
    end
    if g.mana >= 73
        h = deepcopy(g)
        activate_drain!(h)
        push!(states, (h, 73))
    end
    if g.shield_counter <= 1 && g.mana >= 113
        h = deepcopy(g)
        activate_shield!(h)
        push!(states, (h, 113))
    end
    if g.poison_counter <= 1 && g.mana >= 173
        h = deepcopy(g)
        activate_poison!(h)
        push!(states, (h, 173))
    end
    if g.recharge_counter <= 1 && g.mana >= 229
        h = deepcopy(g)
        activate_recharge!(h)
        push!(states, (h, 229))
    end
    states
end

function run_game(g::Game)
    q = PriorityQueue{Game, Int64}()
    dist = Dict(state(g) => 0)
    explored = Set()
    for (v, cost) in next_possible_states(g)
        dist[state(v)] = cost
        enqueue!(q, v, cost)
    end
    while !isempty(q)
        u = dequeue!(q)
        d = dist[state(u)]
        outcome = go_turn!(u)
        if outcome == 1
            return d
        end
        outcome = go_turn!(u)
        if outcome == 1
            return d
        end
        dist[state(u)] = d
        push!(explored, state(u))
        if outcome == 0
            for (v, cost) in next_possible_states(u)
                if !(state(v) in explored)
                    alt = d + cost
                    if state(v) in keys(dist)
                        if alt < dist[state(v)]
                            dist[state(v)] = alt
                        end
                    else
                        dist[state(v)] = alt
                    end
                    enqueue!(q, v, dist[state(v)])
                    push!(explored, state(v))
                end
            end
        end
    end
    return -99
end

run_game (generic function with 1 method)

In [38]:
g1 = Game()
part1 = run_game(g1)
g2 = Game(part=2)
part2 = run_game(g2)
println("Part 1: $part1")
println("part 2: $part2")

Part 1: 1269
part 2: 1309


In [40]:
g1 = Game()
@btime run_game(g1)

  183.797 ms (210431 allocations: 25.66 MiB)


1269

In [41]:
g2 = Game(part=2)
@btime run_game(g2)

  140.572 ms (177501 allocations: 22.22 MiB)


1309